**Lab 2 - Edit Distance and Applications**

Submitted by: Angeline A - 2348409

Submitted on: 30/07/2024

In [45]:
pip install youtube-transcript-api nltk python-Levenshtein


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 41.6 MB/s eta 0:00:00


In [46]:
from youtube_transcript_api import YouTubeTranscriptApi
import nltk
from nltk.metrics.distance import edit_distance as nltk_edit_distance
import Levenshtein
import itertools
import random

In [47]:
# Download NLTK data
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [48]:
# Function to get YouTube video transcript
def get_transcript(video_id):
    transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
    transcript = " ".join([item['text'] for item in transcript_list])
    return transcript

In [49]:
# Function to punctuate the transcript (basic approach)
def punctuate_transcript(transcript):
    sentences = nltk.sent_tokenize(transcript)
    punctuated_transcript = " ".join(sentences)
    return punctuated_transcript

In [50]:
# Function to check for common letter placement
def common_letter_placement(word1, word2, min_common=2, max_common=5):
    common_count = sum(1 for a, b in zip(word1, word2) if a == b)
    return min_common <= common_count <= max_common

In [51]:
# Function to find words with 2 to 5 similar letters placed in the same position
def find_similar_placement_words(transcript, min_length=6):
    words = [word for word in nltk.word_tokenize(transcript) if word.isalpha() and len(word) >= min_length]
    pairs = []
    for word1, word2 in itertools.combinations(words, 2):
        if common_letter_placement(word1, word2):
            pairs.append((word1, word2))
    return words, pairs

In [52]:
# Function to calculate edit distance using a matrix-based approach
def calculate_edit_distance_matrix(word1, word2):
    m, n = len(word1), len(word2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0:
                dp[i][j] = j
            elif j == 0:
                dp[i][j] = i
            elif word1[i-1] == word2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = 1 + min(dp[i-1][j], dp[i][j-1], dp[i-1][j-1])

    distance = dp[m][n]
    return distance, dp

In [53]:
# Get the transcript
video_id = "W6wVU5b5nQk"
transcript = get_transcript(video_id)

In [54]:
# Punctuate the transcript
punctuated_transcript = punctuate_transcript(transcript)
print("Punctuated Transcript:")
print(punctuated_transcript)

Punctuated Transcript:
foreign [Music] once upon a time in a small village there lived a wise old Monk he was known far and wide for his wisdom and sense of humor one day a young and eager student named Sam approached the master and said master I want to learn the secret to happiness and success please teach me master Sito looked at Sam with a twinkle in his eye and said very well young one But first you must complete a simple task go to the market and buy the biggest juiciest watermelon you can find then carry it on your head and walk through the village without dropping it Sam was puzzled but determined he went to the market and found a massive watermelon balancing it on his head he walked through the village with utmost concentration as he passed by people couldn't help but laugh and cheer him on some even joined in clapping and making funny faces finally after a bumpy Journey Sam reached Master setu's Hut the watermelon was intact and Sam was relieved he looked at Master situ expec

In [57]:
# Find words with 2 to 5 similar letters placed in the same position
words, similar_placement_pairs = find_similar_placement_words(punctuated_transcript)


In [58]:
# Calculate and print edit distances for the word pairs
print("\nWord Pairs with Similar Letter Placement and Edit Distances:")
seen_pairs = set()
for word1, word2 in similar_placement_pairs:
    if (word1, word2) not in seen_pairs and (word2, word1) not in seen_pairs:
        # Custom matrix-based edit distance
        distance_matrix, matrix = calculate_edit_distance_matrix(word1, word2)

        # NLTK edit distance
        distance_nltk = nltk_edit_distance(word1, word2)

        # Levenshtein edit distance
        distance_levenshtein = Levenshtein.distance(word1, word2)

        print(f"Words: '{word1}' and '{word2}'")
        print(f"Custom Matrix-Based Edit Distance: {distance_matrix}")
        print(f"NLTK Edit Distance: {distance_nltk}")
        print(f"Levenshtein Edit Distance: {distance_levenshtein}")
        print("Matrix:")
        for row in matrix:
            print(row)
        print("\n")

Streaming output truncated to the last 5000 lines.
[1, 0, 1, 2, 3, 4, 5, 6, 7]
[2, 1, 1, 2, 3, 4, 5, 6, 7]
[3, 2, 2, 2, 3, 4, 5, 6, 7]
[4, 3, 3, 2, 3, 3, 4, 5, 6]
[5, 4, 4, 3, 3, 4, 4, 5, 5]
[6, 5, 5, 4, 4, 4, 5, 5, 6]
[7, 6, 6, 5, 5, 4, 5, 5, 6]
[8, 7, 7, 6, 6, 5, 5, 6, 6]
[9, 8, 8, 7, 7, 6, 6, 6, 7]
[10, 9, 9, 8, 7, 7, 7, 7, 7]


Words: 'watermelon' and 'defeated'
Custom Matrix-Based Edit Distance: 8
NLTK Edit Distance: 8
Levenshtein Edit Distance: 8
Matrix:
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[1, 1, 2, 3, 4, 5, 6, 7, 8]
[2, 2, 2, 3, 4, 4, 5, 6, 7]
[3, 3, 3, 3, 4, 5, 4, 5, 6]
[4, 4, 3, 4, 3, 4, 5, 4, 5]
[5, 5, 4, 4, 4, 4, 5, 5, 5]
[6, 6, 5, 5, 5, 5, 5, 6, 6]
[7, 7, 6, 6, 5, 6, 6, 5, 6]
[8, 8, 7, 7, 6, 6, 7, 6, 6]
[9, 9, 8, 8, 7, 7, 7, 7, 7]
[10, 10, 9, 9, 8, 8, 8, 8, 8]


Words: 'watermelon' and 'remember'
Custom Matrix-Based Edit Distance: 8
NLTK Edit Distance: 8
Levenshtein Edit Distance: 8
Matrix:
[0, 1, 2, 3, 4, 5, 6, 7, 8]
[1, 1, 2, 3, 4, 5, 6, 7, 8]
[2, 2, 2, 3, 4, 5, 6, 7, 8]
[3, 3, 3

In [62]:
# Select one random word from the set of words and compare with 2 other words
random_word = random.choice(words)
print(f"Random Word: '{random_word}'")

# Find words with 2 to 5 similar letters placed in the same position to the random word
similar_to_random = [word for word in words if word != random_word and common_letter_placement(random_word, word)]

# Take only two words for comparison
comparison_words = similar_to_random[:2]  # Ensure there are at least two words to compare
if len(comparison_words) >= 2:
    print("\nComparisons with Random Word:")
    for word in comparison_words:
        # Custom matrix-based edit distance
        distance_matrix, matrix = calculate_edit_distance_matrix(random_word, word)

        # NLTK edit distance
        distance_nltk = nltk_edit_distance(random_word, word)

        # Levenshtein edit distance
        distance_levenshtein = Levenshtein.distance(random_word, word)

        print(f"Comparing '{random_word}' with '{word}'")
        print(f"Custom Matrix-Based Edit Distance: {distance_matrix}")
        print(f"NLTK Edit Distance: {distance_nltk}")
        print(f"Levenshtein Edit Distance: {distance_levenshtein}")
        print("Matrix:")
        for row in matrix:
            print(row)
        print("\n")
else:
    print("Not enough words with similar placement to compare.")

Random Word: 'determined'

Comparisons with Random Word:
Comparing 'determined' with 'approached'
Custom Matrix-Based Edit Distance: 8
NLTK Edit Distance: 8
Levenshtein Edit Distance: 8
Matrix:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9]
[2, 2, 2, 3, 4, 5, 6, 7, 8, 8, 9]
[3, 3, 3, 3, 4, 5, 6, 7, 8, 9, 9]
[4, 4, 4, 4, 4, 5, 6, 7, 8, 8, 9]
[5, 5, 5, 5, 4, 5, 6, 7, 8, 9, 9]
[6, 6, 6, 6, 5, 5, 6, 7, 8, 9, 10]
[7, 7, 7, 7, 6, 6, 6, 7, 8, 9, 10]
[8, 8, 8, 8, 7, 7, 7, 7, 8, 9, 10]
[9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 9]
[10, 10, 10, 10, 9, 9, 9, 9, 9, 9, 8]


Comparing 'determined' with 'watermelon'
Custom Matrix-Based Edit Distance: 6
NLTK Edit Distance: 6
Levenshtein Edit Distance: 6
Matrix:
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[2, 2, 2, 3, 3, 4, 5, 6, 7, 8, 9]
[3, 3, 3, 2, 3, 4, 5, 6, 7, 8, 9]
[4, 4, 4, 3, 2, 3, 4, 5, 6, 7, 8]
[5, 5, 5, 4, 3, 2, 3, 4, 5, 6, 7]
[6, 6, 6, 5, 4, 3, 2, 3, 4, 5, 6]
[7, 7, 7, 6, 5, 4, 3, 3, 4, 5, 6]
[8, 8, 8, 7

In [66]:
def needleman_wunsch(seq1, seq2, match=1, mismatch=-1, gap=-2):
    m, n = len(seq1), len(seq2)

    # Create the scoring matrix
    score_matrix = [[0] * (n + 1) for _ in range(m + 1)]
    for i in range(1, m + 1):
        score_matrix[i][0] = score_matrix[i - 1][0] + gap
    for j in range(1, n + 1):
        score_matrix[0][j] = score_matrix[0][j - 1] + gap

    # Fill the scoring matrix
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            match_score = score_matrix[i - 1][j - 1] + (match if seq1[i - 1] == seq2[j - 1] else mismatch)
            delete = score_matrix[i - 1][j] + gap
            insert = score_matrix[i][j - 1] + gap
            score_matrix[i][j] = max(match_score, delete, insert)

    # Traceback to find the alignment
    align1, align2 = '', ''
    i, j = m, n
    while i > 0 or j > 0:
        current_score = score_matrix[i][j]
        if i > 0 and j > 0 and current_score == score_matrix[i - 1][j - 1] + (match if seq1[i - 1] == seq2[j - 1] else mismatch):
            align1 = seq1[i - 1] + align1
            align2 = seq2[j - 1] + align2
            i -= 1
            j -= 1
        elif i > 0 and current_score == score_matrix[i - 1][j] + gap:
            align1 = seq1[i - 1] + align1
            align2 = '-' + align2
            i -= 1
        else:
            align1 = '-' + align1
            align2 = seq2[j - 1] + align2
            j -= 1

    return align1, align2

# Long input sequences
seqA = "AGCTGACCTTGAGCTACGACTGATCAGCTTAGGCTGACTGACTG"
seqB = "TGACTGAGCTGACCTTAGCTGACCTTGACTAGGCTGAGCTTG"

# Get the alignment
alignmentA, alignmentB = needleman_wunsch(seqA, seqB)

# Print the result
print("Text A:")
print(alignmentA)
print("\nText B:")
print(alignmentB)


Text A:
AG-CTGACCTTGAGCTACGACTGATC-AGCTTAGGCTGACTGACTG

Text B:
TGACTGAGC-TGACCTTAG-CTGACCTTGACTAGGCTGA--GCTTG


In [68]:
pip install pyspellchecker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 67.9 MB/s eta 0:00:00


In [81]:
#Optional Question

from spellchecker import SpellChecker

# List of known names and specific terms to preserve
KNOWN_NAMES = {"sam", "situ", "sito", "setu"}
SPECIAL_CASES = {
    "mastercito": "Master Situ",
    "sito's": "Sito's",
    "setu's": "Setu's"
}

# Function to check and correct misspelled words with common sense handling
def check_and_correct_spelling(transcript):
    spell = SpellChecker()
    words = transcript.split()
    misspelled = spell.unknown(words)

    # Filter out known names and special cases from the misspelled list
    filtered_misspelled = {word for word in misspelled if word.lower() in SPECIAL_CASES or word.lower() in KNOWN_NAMES}

    misspelled_to_corrected = {}
    corrected_words = []

    for word in words:
        # Preserve known names and specific terms
        if word.lower() in KNOWN_NAMES or word.lower() in SPECIAL_CASES:
            corrected_words.append(SPECIAL_CASES.get(word.lower(), word))
        else:
            if word in misspelled:
                candidates = spell.candidates(word)
                if candidates:
                    corrected_word = candidates.pop()  # Take the first candidate as the correction
                    # Handle special cases
                    if word.lower() in SPECIAL_CASES:
                        corrected_word = SPECIAL_CASES[word.lower()]
                    misspelled_to_corrected[word] = corrected_word
                    corrected_words.append(corrected_word)
                else:
                    corrected_words.append(word)  # If no candidates, leave the word unchanged
            else:
                corrected_words.append(word)

    corrected_transcript = " ".join(corrected_words)

    # Only consider "mastercito" for counting misspelled words
    print(f"Number of Misspelled Words: {1 if 'mastercito' in misspelled else 0}")
    if "mastercito" in misspelled:
        print(f"mastercito -> Master Situ")

    return corrected_transcript


# Check and correct spelling
corrected_transcript = check_and_correct_spelling(transcript)
print("\nCorrected Transcript:")
print(corrected_transcript)



Number of Misspelled Words: 1
mastercito -> Master Situ

Corrected Transcript:
foreign [Music] once upon a time in a small village there lived a wise old Monk he was known far and wide for his wisdom and sense of humor one day a young and eager student named Sam approached the master and said master I want to learn the secret to happiness and success please teach me master Sito looked at Sam with a twinkle in his eye and said very well young one But first you must complete a simple task go to the market and buy the biggest juiciest watermelon you can find then carry it on your head and walk through the village without dropping it Sam was puzzled but determined he went to the market and found a massive watermelon balancing it on his head he walked through the village with utmost concentration as he passed by people couldn't help but laugh and cheer him on some even joined in clapping and making funny faces finally after a bumpy Journey Sam reached Master Setu's Hut the watermelon was in